In [4]:
!pip install transformers -U


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [6]:
data = '/content/train.csv'
df = pd.read_csv(data)
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
df['toxic'].value_counts()

,count
toxic,
0,144277
1,15294


In [8]:
# df = df[['comment_text','toxic']]
# df = df[0:1000]
# data.head()

# Bert

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
import torch
print(torch.cuda.is_available())  # Should print True if GPU is available

True


In [12]:
model = model.to('cuda')

In [13]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [14]:
X = list(df["comment_text"])
y = list(df["toxic"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)

X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [15]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [16]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [17]:
len(X_train_tokenized['attention_mask'][0])

512

In [18]:
len(X_train),len(X_val)

(127656, 31915)

In [19]:

# Create torch dataset

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [20]:

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [21]:
train_dataset[5]


{'input_ids': tensor([  101,  5665, 20734, 17873,  2061,  2952, 18454,  8585,  2001,  3018,
          1037,  3101,  1999,  2244,  3925,  2005,  2824,  2008,  2165,  2173,
          1999,  2255,  3925,  1029,  2071,  3531,  2619, 17824,  2006,  1996,
          3043,  6149,  2023,  1029,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [22]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tajuar-akash (tajuar-akash-ccn-univesity) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.218200


In [ ]:
trainer.evaluate()

In [ ]:
np.set_printoptions(suppress=True)

#The line np.set_printoptions(suppress=True) is a command in NumPy (imported as np) used to control the way NumPy arrays are printed in Python.


In [ ]:
text = "That was good point"
# text = "go to hell"

inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

predictions = predictions.cpu().detach().numpy()
predictions

In [ ]:
trainer.save_model('CustomModel')


In [ ]:
model_2 = BertForSequenceClassification.from_pretrained("CustomModel")
model_2.to('cuda')

In [ ]:

# text = "That was good point"
text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

In [ ]:

# trainer.save_model('/content/drive/MyDrive/Youtube Tutorials/toxic')
# model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Youtube Tutorials/toxic")
# model_2.to('cuda')

Let's explain **padding** in simple terms!

Imagine you're making a list of words, but each list needs to have the same number of words, even if some of the lists are shorter. So, to make them all the same length, you add empty spaces or extra words to the shorter lists. These empty spaces are like **padding**.

Now, let's look at your example:

```python
sample_data = ["I am eating", "I am playing"]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)
```

### What is happening in this example?

1. **Padding**:
   - "I am eating" and "I am playing" are both short sentences. But let's say we want all sentences to be the same length, like making sure every sentence has 512 words.
   - If the sentence has fewer words, we "pad" it by adding some empty spaces or special tokens (like `[PAD]`) to make it the same length as others.

2. **Truncation**:
   - If the sentence is too long (longer than the maximum length allowed), we cut off the extra words. This is **truncation**.

3. **max_length=512**:
   - This means we're setting a limit on the sentence length. If the sentence is too short, we add padding; if it's too long, we cut it off. The longest sentence can have up to 512 words.

### Simple Example:
- "I am eating" would become:  
  `["I", "am", "eating", "<PAD>", "<PAD>", ..., "<PAD>"]` (adding padding until it has 512 words).
  
- "I am playing" would also be padded similarly.

This way, all sentences are the same length, and the model can handle them more easily!

Does this help explain padding? Let me know if you need more clarification!

Sure! Let's break it down simply.

### What is **Attention Mask**?

An **attention mask** is a mechanism used in models like BERT to tell the model which tokens (words or sub-words) it should pay attention to and which tokens are just **padding tokens** that should be ignored.

When you use padding to make all sentences the same length, some tokens are just there to fill space (these are padding tokens). The model doesn't need to process or "pay attention" to these padding tokens.

The **attention mask** is a binary array (0s and 1s) that indicates which tokens the model should **pay attention** to (1) and which it should **ignore** (0).

### How Does the Attention Mask Work?

- **1** means "pay attention to this token."
- **0** means "ignore this token because it's just padding."

### Example:

Let’s take an example sentence and see how padding and the attention mask work together.

#### Input Sentence:
`"I am eating"`

After tokenization, let’s say it looks like this:
```
["I", "am", "eating", "[PAD]", "[PAD]"]
```

In this case, we padded the sentence to make it a fixed length, say 5 tokens. Now, the **attention mask** for this sequence would look like this:
```
[1, 1, 1, 0, 0]
```

Here:
- The `1`s indicate that the model should pay attention to the actual words: "I", "am", and "eating".
- The `0`s indicate that the model should ignore the padding tokens (`[PAD]`).

### Code Example:
If you run:

```python
print(X_train_tokenized['attention_mask'][0])
```

It will print the attention mask for the first tokenized sentence in your `X_train_tokenized` dataset.

- **Example Output**: If the sentence had 3 tokens and 2 padding tokens, it might print something like:
  ```
  [1, 1, 1, 0, 0]
  ```

This tells the model:
- Look at the first 3 tokens (real words).
- Ignore the last 2 tokens (padding).

### Summary:
The **attention mask** helps the model focus only on the real tokens (words) and ignore the padding tokens, making the model's processing more efficient and accurate.

Let me know if you'd like more details on how this works!

This code defines a custom dataset class for PyTorch, which will be used to load data in a format that can be passed to a machine learning model. Here's a simple explanation:

### Code Breakdown:

1. **Class Definition**:  
   `class Dataset(torch.utils.data.Dataset):`
   - This creates a custom dataset class that inherits from `torch.utils.data.Dataset`. This allows you to easily handle your data in batches for training.

2. **`__init__` Method**:  
   ```python
   def __init__(self, encodings, labels=None):
       self.encodings = encodings
       self.labels = labels
   ```
   - **Purpose**: Initializes the dataset with two arguments:
     - `encodings`: This contains the tokenized input data (like word tokens).
     - `labels`: These are the labels (or targets) for the data, used for supervised learning (like classifications).

3. **`__getitem__` Method**:  
   ```python
   def __getitem__(self, idx):
       item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
       if self.labels:
           item["labels"] = torch.tensor(self.labels[idx])
       return item
   ```
   - **Purpose**: Fetches a single item (input and label) from the dataset at the specified index (`idx`).
   - **Explanation**:
     - For each key-value pair in `self.encodings`, it converts the value at the index `idx` into a PyTorch tensor.
     - If `labels` exist, it adds the label at index `idx` to the item.
   - It returns the data (`item`), which contains both the input data and label.

4. **`__len__` Method**:  
   ```python
   def __len__(self):
       return len(self.encodings["input_ids"])
   ```
   - **Purpose**: Returns the number of data points (samples) in the dataset.
   - **Explanation**: It checks the length of the `input_ids` from the `encodings` dictionary, which holds the number of tokenized inputs.

### Summary:
This class is a wrapper around the input data (`encodings`) and the labels, allowing PyTorch to load data in batches for training. It handles:
- Converting data into tensors.
- Fetching individual samples and their corresponding labels.
- Returning the length of the dataset.

Let me know if you need further clarification!